In [1]:
import time
import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException,ElementClickInterceptedException,StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import re,os,requests
import subprocess
import financedatabase as fd
from io import StringIO
import threading
import pandas as pd

getting tickers and meta data

In [2]:
equities = fd.Equities()
equities_united_states = equities.select(country="United States")
df = equities_united_states.query('market_cap.isin(["Mega Cap","Large Cap","Mid Cap","Small Cap","Micro Cap"])')

In [3]:
equities_united_states['sector'].unique()

array(['Health Care', 'Materials', 'Financials', 'Industrials',
       'Consumer Staples', 'Real Estate', nan, 'Consumer Discretionary',
       'Information Technology', 'Communication Services', 'Energy',
       'Utilities'], dtype=object)

In [4]:
sector='Communication Services'
tickers=list(equities_united_states.query(f"sector=='{sector}'").index)

In [5]:
from selenium import webdriver
from selenium_stealth import stealth



class Browser:
    def __init__(self, options:str ,open_on: str = '',undetected: bool =False):
        print('Launching Browser.')
        self.browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        if open_on:
            self.open_page(url=open_on)
    
    def open_page(self, url: str):
        print(f"Opening {url}.")
        self.browser.get(url)

    def close_browser(self):
        print('Closing Browser.')
        self.browser.close()
        
    def click_button(self, by: By, value: str):
        tries=0
        while tries<100:
            try:
                button = self.browser.find_element(by=by, value=value)
                button.click()
                time.sleep(0.5)
                break
            except NoSuchElementException as e:
                tries+=1
                print(f'retrying {tries}')
                    
        else:
            raise e
    def download_csv(self):
        button_xpath='//*[@id="main"]/div[2]/div[1]/div[3]/div[1]/div/div[2]'
        download_xpath='//*[@id="main"]/div[2]/div[1]/div[3]/div[1]/div/div[2]/div/button[4]'
        self.click_button(by=By.XPATH,value=button_xpath)
        self.click_button(by=By.XPATH,value=download_xpath)
        


In [6]:
def FetchTable(browser):
    
    def element_exists_by_xpath(xpath):
        try:
            browser.browser.find_element(by=By.XPATH,value=xpath)
        except NoSuchElementException:
            return False
        return True
    
    def fetch_by_xpath(xpath):
        try:
            txt=browser.browser.find_element(by=By.XPATH,value=xpath).text
        except StaleElementReferenceException:
            time.sleep(0.1)
            txt=browser.browser.find_element(by=By.XPATH,value=xpath).text
            
        if isinstance(txt,str):
            txt=re.sub(r'(?<= \d{4}) ',";",txt.replace('Date Value\n',''))
        return txt
    
    Table=''
    tries=0
    pages=30
    next_button_xpath= f'//*[@id="ycn-historical-data-table-0"]/div[2]/div/div[7]'
    data_xpath='//*[@id="ycn-historical-data-table-0"]/div[3]/div'
    page_number_xpath = f'//*[@id="ycn-historical-data-table-0"]/div[2]/div/div[1]'
    while tries<10:
        try:
            pages=int(re.search(r'\d*$',browser.browser.find_element(by=By.XPATH,value=page_number_xpath).text)[0])+1
            break
        except Exception:
            tries+=1

    for page in range(1,pages):
        print('fetshing data on page '+str(page)+f" out of {pages-1}")
        if element_exists_by_xpath(data_xpath):
            txt_page=fetch_by_xpath(data_xpath)
            while (txt_page in Table):
                time.sleep(1)
                txt_page=fetch_by_xpath(data_xpath)
                
            Table=f"{Table}{txt_page}\n"
            print(len(Table))    
            # if txt_page not in Table:
            #     Table=f"{Table}{txt_page}\n"
            #     print("copied")
            try:
                browser.click_button(by=By.XPATH,value=next_button_xpath)
                print('Clickable and clicked')
            except ElementClickInterceptedException:
                if txt_page not in Table:
                    Table=f"{Table}{txt_page}\n"
    return Table.replace('Date Value\n','')

In [7]:
def check_Ycharts(browser,ticker):
    browser.open_page(url=f"https://ycharts.com/companies/{ticker}/market_cap")
    time.sleep(0.2)
    if 'Page Not Found' in browser.browser.page_source:
        print('No Data found for'+ticker)
        return False
    if  "Market Cap" not in browser.browser.find_element(by=By.XPATH,value=f'/html/body/main/div/div[3]/div/div/div/div[1]/div/h1').text:
        return False
    Table=FetchTable(browser)
    if not Table:
        return False
    df=pd.DataFrame(list(map(lambda x: x.split(';'),Table.split('\n'))),columns = ['Date', 'Market Cap']).set_index('Date').dropna().applymap(lambda x: float(x[:-1])*10**12 if x.endswith("T") else float(x[:-1])*10**9 if x.endswith("B") else float(x[:-1])*10**6  if x.endswith("M") else float(x[:-1])*10**3 )
    df.index=pd.to_datetime(df.index)
    df.to_csv(fr'Data\Stockanalysis.com\{ticker}-market-cap.csv')
    return True

In [8]:
import os

# Set the path to the Data folder
def list_done():
    data_folder_path = f'Data\Stockanalysis.com\{sector}\Stocks'

# Create an empty list to hold the tickers
    Done = []

# Loop through all the files in the Data folder
    for filename in os.listdir(data_folder_path):
    # Check if the file is a CSV file
        if filename.endswith('.csv') and ('-' in filename):
        # Get the ticker from the filename
            ticker = filename.split('-')[0].upper()
        
        # Add the ticker to the list
            Done.append(ticker)
    return Done

Done=list_done()



In [9]:
skiped=[]

In [10]:
Ychart_tickers=[]

In [11]:
from IPython.display import clear_output

def first_process(browser,Ychart_tickers,skiped):
    Tickers_to_fetch = list(set(tickers)-set(list_done())-set(skiped))
    for k,ticker in enumerate(Tickers_to_fetch):
        button_xpath='//*[@id="main"]/div[2]/div[1]/div[4]/div/div[1]/div[2]/div[2]/button'
        download_xpath='//*[@id="main"]/div[2]/div[1]/div[4]/div/div[1]/div[2]/div[2]/div/button[2]'
        check_tries=0
        browser.open_page(f"https://stockanalysis.com/stocks/{ticker}/market-cap/")
        time.sleep(0.5)
        while ('Just a moment...' in browser.browser.page_source) and check_tries<3000:
            browser.open_page(f"https://stockanalysis.com/stocks/{ticker}/market-cap/")
            check_tries+=1
            time.sleep(5*check_tries)
            browser.browser.execute_script(f'''window.open("https://stockanalysis.com/stocks/{ticker}/market-cap/","_blank");''')
            browser.browser.close()
            browser.browser.switch_to.window(window_name=browser.browser.window_handles[-1])
        if (not 'Page Not Found - 404' in browser.browser.page_source) and (not "No market cap history was found for this stock." in browser.browser.page_source):
            # try:
                browser.click_button(by=By.XPATH,value=button_xpath)
                time.sleep(0.1)
                browser.click_button(by=By.XPATH,value=download_xpath)
                time.sleep(0.1)
            # except Exception:
            #     skiped.append(ticker)
        else:
            if ticker not in Ychart_tickers:
                Ychart_tickers.append(ticker)

        clear_output(wait=True)
        print(f'{(k+1)/len(tickers):.2f} % done and {len(skiped)} ({len(skiped)/len(Tickers_to_fetch):.2f}%) skipped')
    Ychart_tickers.append('Terminate Process')

def second_process(browser,Ychart_tickers,skiped):
    
    while True:
        clear_output(wait=True)

        if Ychart_tickers:
            for ticker in Ychart_tickers:
                if ticker=='Terminate Process':
                    return
                if not check_Ycharts(browser,ticker):     
                    skiped.append(ticker)
                Ychart_tickers.remove(ticker)
        else:
            print("list is empty checking again")
            time.sleep(1)
    
    

In [12]:
options1 = webdriver.ChromeOptions()
options1.add_argument("--user-data-dir=C:/Users/amine/AppData/Local/Google/Chrome/User Data/Profile 2/")
options2 = webdriver.ChromeOptions()
options2.add_argument("--user-data-dir=C:/Users/amine/AppData/Local/Google/Chrome/User Data/Profile 3/")
options3 = webdriver.ChromeOptions()
options3.add_argument("--user-data-dir=C:/Users/amine/AppData/Local/Google/Chrome/User Data/Profile 4/")

In [37]:
options = webdriver.ChromeOptions()
options.add_argument("--user-data-dir=C:/Users/amine/AppData/Local/Google/Chrome/User Data/Profile 1/")

browser = Browser(options)
# browser1 = Browser(options1)


Launching Browser.


In [35]:
# t1 = threading.Thread(target=first_process, args=(browser,Ychart_tickers,skiped))
t2 = threading.Thread(target=second_process, args=(browser1,Ychart_tickers,skiped))

# starting thread 1
# t1.start()
# starting thread 2
t2.start()

# wait until thread 1 is completely executed
# t1.join()
# wait until thread 2 is completely executed
t2.join()


Opening https://ycharts.com/companies/ADVWW/market_cap.
No Data found forADVWW


In [36]:
Ychart_tickers

['Terminate Process']

In [26]:
class ScrapeThread(threading.Thread):
    def __init__(self,List_Tickers,Results,options,skiped=[]):
        threading.Thread.__init__(self)
        self.Results=Results
        self.options = options
        self.List_Tickers = List_Tickers
        self.skiped=skiped
    
    def run(self):
        self.browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=self.options)
        try:
            self.SCRAPPING(self.List_Tickers)
        except BaseException as e:
            self.exc = e
        # time.wait()
    def join(self, timeout=None):
        super(ScrapeThread, self).join(timeout)
        if self.exc:
            raise self.exc
        return self.ret
    
    def open_page(self, url: str,new_tab=False):
        
        print(f"Opening {url}.")
        if new_tab:
            self.browser.execute_script(f'''window.open("{url}","_blank");''')
        else:
            self.browser.get(url)
  
    def click_button(self, by: By, value: str):
        button = self.browser.find_element(by=by, value=value)
        button.click()
    
    def is_clickable(self, by: By, value: str):
        try:
            # Wait up to 10 seconds for the element to be clickable
            element = WebDriverWait(self.browser, 5).until(EC.element_to_be_clickable((by, value)))
            return True
        except:
            return False
    
    def SCRAPPING(self,Tickers):
        MC={}
        if Tickers is None:
            Tickers=equities_united_states.query("sector=='Utilities'").index
        Tickers=list(set(Tickers)- set(self.Results.keys())-set(self.skiped))
        new_window=False
        # self.browser.switch_to.window(window_name=self.browser.window_handles[-1])
        # if new_window:
        #     self.browser.switch_to.new_window('window')
        if isinstance(Tickers,str):
            Tickers=[Tickers]
        for i,ticker in enumerate(Tickers):
            self.open_page(url=f"https://ycharts.com/companies/{ticker}/market_cap")

            if 'Page Not Found' in self.browser.page_source:
                print('No Data found for'+ticker)
                self.skiped.append(ticker)
                continue
            if  "Market Cap" not in self.browser.find_element(by=By.XPATH,value=f'/html/body/main/div/div[3]/div/div/div/div[1]/div/h1').text:
                self.skiped.append(ticker)
                continue
            Table=self.FetchTable()
            if not Table:
                self.skiped.append(ticker)
                continue
            MC[ticker]=pd.DataFrame(list(map(lambda x: x.split(';'),Table.split('\n'))),columns = ['Date', 'Market Cap']).set_index('Date').dropna().applymap(lambda x: float(x[:-1])*10**12 if x.endswith("T") else float(x[:-1])*10**9 if x.endswith("B") else float(x[:-1])*10**6  if x.endswith("M") else float(x[:-1])*10**3 )
            MC[ticker].index=pd.to_datetime(MC[ticker].index)
            MC[ticker].reset_index(inplace=True)
            if not i%5:
                self.Results.update(MC)
            self.Results.update(MC)

    def FetchTable(self):
        
        def element_exists_by_xpath(xpath):
            try:
                self.browser.find_element(by=By.XPATH,value=xpath)
            except NoSuchElementException:
                return False
            return True
        
        def fetch_by_xpath(xpath):
            try:
                txt=self.browser.find_element(by=By.XPATH,value=xpath).text
            except StaleElementReferenceException:
                time.sleep(0.1)
                txt=self.browser.find_element(by=By.XPATH,value=xpath).text
                
            if isinstance(txt,str):
                txt=re.sub(r'(?<= \d{4}) ',";",txt.replace('Date Value\n',''))
            return txt

        Table=''
        tries=0
        pages=30
        next_button_xpath= f'//*[@id="ycn-historical-data-table-0"]/div[2]/div/div[7]'
        data_xpath='//*[@id="ycn-historical-data-table-0"]/div[3]/div'
        page_number_xpath = f'//*[@id="ycn-historical-data-table-0"]/div[2]/div/div[1]'
        while tries<10:
            try:
                pages=int(re.search(r'\d*$',self.browser.find_element(by=By.XPATH,value=page_number_xpath).text)[0])+1
                break
            except Exception:
                tries+=1

        for page in range(1,pages):
            print('fetshing data on page '+str(page)+f" out of {pages-1}")
            if element_exists_by_xpath(data_xpath):
                txt_page=fetch_by_xpath(data_xpath)
                while (txt_page in Table):
                    time.sleep(1)
                    txt_page=fetch_by_xpath(data_xpath)
                    
                Table=f"{Table}{txt_page}\n"
                print(len(Table))    
                # if txt_page not in Table:
                #     Table=f"{Table}{txt_page}\n"
                #     print("copied")
                try:
                    self.click_button(by=By.XPATH,value=next_button_xpath)
                    print('Clickable and clicked')
                except ElementClickInterceptedException:
                    if txt_page not in Table:
                        Table=f"{Table}{txt_page}\n"
        return Table.replace('Date Value\n','')

In [27]:
class ScrapeThreadv2(threading.Thread):
    def __init__(self,List_Tickers,Results,options,skiped=[]):
        threading.Thread.__init__(self)
        self.Results=Results
        self.options = options
        self.List_Tickers = List_Tickers
        self.skiped=skiped
    
    def run(self):
        self.browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=self.options)
        try:
            self.SCRAPPING(self.List_Tickers)
        except BaseException as e:
            self.exc = e
        # time.wait()
    def join(self, timeout=None):
        super(ScrapeThread, self).join(timeout)
        if self.exc:
            raise self.exc
        return self.ret
    
    def open_page(self, url: str,new_tab=False):
        
        print(f"Opening {url}.")
        if new_tab:
            self.browser.execute_script(f'''window.open("{url}","_blank");''')
        else:
            self.browser.get(url)
  
    def click_button(self, by: By, value: str):
        button = self.browser.find_element(by=by, value=value)
        button.click()
    
    def is_clickable(self, by: By, value: str):
        try:
            # Wait up to 10 seconds for the element to be clickable
            element = WebDriverWait(self.browser, 5).until(EC.element_to_be_clickable((by, value)))
            return True
        except:
            return False
    
    def download_csv(self):
        button_xpath='//*[@id="main"]/div[2]/div[1]/div[3]/div[1]/div/div[2]'
        download_xpath='//*[@id="main"]/div[2]/div[1]/div[3]/div[1]/div/div[2]/div/button[4]'
        self.click_button(by=By.XPATH,value=button_xpath)
        self.click_button(by=By.XPATH,value=download_xpath)
    
    

In [190]:
Results={}
sector='Financials'


In [191]:
# %store Results
# %store -r Results
# %store skiped
# %store -r skiped

my_list = equities_united_states.query(f'sector=="{sector}"').index


# Determine the length of each part
part_length = len(my_list) // 4

Test_tickers = [
	["TSLA"],
	['A'],
	['AAPL'],
	['DIS']
	# my_list[:part_length],
	# my_list[part_length:part_length*2],
	# my_list[part_length*2:part_length*3],
	# my_list[part_length*3:]
]


# Options = [webdriver.ChromeOptions()]*len(Test_tickers)



Options = [options,options1,options2,options3]

threads = []
for i,Tickers in enumerate(Test_tickers):
	t = ScrapeThread(Tickers,Results,Options[i])
	t.start()
	threads.append(t)

for t in threads:
	t.join()	
 
cc_MC=dict(zip(Results.keys(),list(map(lambda x: x.set_index("Date") ,Results.values()))))
pd.concat(cc_MC,axis=1).to_csv(fr'Data\Scraped MarketCap\{sector}.csv')


Opening https://ycharts.com/companies/DIS/market_cap.
Opening https://ycharts.com/companies/A/market_cap.


Exception in thread Thread-32:
Traceback (most recent call last):
  File "C:\Users\amine\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\amine\AppData\Local\Temp\ipykernel_19708\4100611930.py", line 10, in run
  File "c:\Users\amine\OneDrive\Documents\PFA\.venv\lib\site-packages\selenium\webdriver\chrome\webdriver.py", line 84, in __init__
    super().__init__(
  File "c:\Users\amine\OneDrive\Documents\PFA\.venv\lib\site-packages\selenium\webdriver\chromium\webdriver.py", line 104, in __init__
    super().__init__(
  File "c:\Users\amine\OneDrive\Documents\PFA\.venv\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 286, in __init__


Opening https://ycharts.com/companies/AAPL/market_cap.


    self.start_session(capabilities, browser_profile)
  File "c:\Users\amine\OneDrive\Documents\PFA\.venv\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 378, in start_session
    response = self.execute(Command.NEW_SESSION, parameters)
  File "c:\Users\amine\OneDrive\Documents\PFA\.venv\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 440, in execute
    self.error_handler.check_response(response)
  File "c:\Users\amine\OneDrive\Documents\PFA\.venv\lib\site-packages\selenium\webdriver\remote\errorhandler.py", line 245, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.WebDriverException: Message: unknown error: Chrome failed to start: exited normally.
  (unknown error: DevToolsActivePort file doesn't exist)
  (The process started from chrome location C:\Program Files\Google\Chrome\Application\chrome.exe is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
Backtrace:
	Get

AttributeError: 'ScrapeThread' object has no attribute 'exc'

In [ ]:
list(Results.keys())[-10:]

['OSPN', 'MCAA', 'UNIB', 'MFDB', 'FITB', 'PMVC', 'SSBI', 'TOWN', 'FRBK', 'OPY']

In [ ]:
len(Results)

279

In [ ]:

cc_MC=dict(zip(Results.keys(),list(map(lambda x: x.set_index("Date") ,Results.values()))))

In [ ]:
pd.concat(cc_MC,axis=1).to_csv(fr'Data\Scraped MarketCap\{sector}.csv')


In [ ]:
Browser(options,)


Launching Browser.


In [ ]:
Browser(options1,)

Launching Browser.


In [ ]:
%store MC
corrected_MC=dict(zip(MC.keys(),list(map(lambda x: x[['Date','Market Cap']],MC.values()))))
pd.concat(MC,axis=1).to_csv('Data/Scraped MarketCap/NON-Utilities.csv')

Stored 'MC' (dict)


In [ ]:

Tickers=tickers
new_window=True
browser.browser.switch_to.window(window_name=browser.browser.window_handles[-1])
if new_window:
    browser.browser.switch_to.new_window('window')
if isinstance(Tickers,str):
    Tickers=[Tickers]
MC={}
for ticker in Tickers:
    browser.open_page(url=f"https://ycharts.com/companies/{ticker}/market_cap")
    
    if 'Page Not Found' in browser.browser.page_source:
        logs('No Data found for'+ticker)
        end_logs()
        continue
    Table=browser.FetchTable()
    if not Table:
        continue
    MC[ticker]=pd.DataFrame(list(map(lambda x: x.split(';'),Table.split('\n'))),columns = ['Date', 'Market Cap']).set_index('Date').dropna().applymap(lambda x: float(x[:-1])*10**9 if x.endswith("B") else float(x[:-1])*10**6  if x.endswith("M") else float(x[:-1])*10**3 )
    MC[ticker].index=pd.to_datetime(MC[ticker].index)
Results.append(MC)

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=112.0.5615.138)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0082DCE3+50899]
	(No symbol) [0x007BE111]
	(No symbol) [0x006C5588]
	(No symbol) [0x006B70CB]
	(No symbol) [0x006B778A]
	(No symbol) [0x006B7728]
	(No symbol) [0x006AC37A]
	(No symbol) [0x006ACBD7]
	(No symbol) [0x007144AB]
	(No symbol) [0x0070B8C3]
	(No symbol) [0x006E7708]
	(No symbol) [0x006E886D]
	GetHandleVerifier [0x00A93EAE+2566302]
	GetHandleVerifier [0x00AC92B1+2784417]
	GetHandleVerifier [0x00AC327C+2759788]
	GetHandleVerifier [0x008C5740+672048]
	(No symbol) [0x007C8872]
	(No symbol) [0x007C41C8]
	(No symbol) [0x007C42AB]
	(No symbol) [0x007B71B7]
	BaseThreadInitThunk [0x767D0099+25]
	RtlGetAppContainerNamedObjectPath [0x77117B6E+286]
	RtlGetAppContainerNamedObjectPath [0x77117B3E+238]


In [ ]:
from functools import reduce

reduce(lambda  left,right: pd.merge(left,right,on=['DATE'],
                                            how='outer'), MC.values())

KeyError: 'DATE'

In [ ]:
browser.browser.find_element(By.TAG_NAME ,'body').send_keys(Keys.COMMAND + 't') 

In [ ]:
window_name = browser.browser.window_handles[-1]

In [ ]:
browser.browser.switch_to.window(window_name=window_name)

In [ ]:
browser.open_page(url=f"https://ycharts.com/companies/{ticker}/market_cap")